In [3]:
# finetune_mindflow_ulysses.py
import os
import json
import time
import torch
import random
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast
from peft import LoraConfig, get_peft_model
import torch.nn as nn

# -------------------------
# CONFIG - 请按需修改
# -------------------------
CONFIG = {
    "base_model_path": "/root/autodl-tmp/Pro/qwen2.5-sft-full",   # 模型一：已微调的诗歌模型（基座）
    "ulysses_txt": "/root/autodl-tmp/Pro/Ulysses尤利西斯.txt",    # 尤利西斯原文 txt（本地）
    "output_dir": "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses",# 输出模型（二）
    "max_len": 1024,      # 单个样本最大 token 长度（可调：1024/2048）
    "stride": 512,        # 滑动窗口重叠
    "batch_size": 1,      # per device batch size（通常 1 或 2）
    "gradient_accumulation_steps": 4,
    "learning_rate": 2e-5,
    "num_train_epochs": 2,
    "fp16": True,
    "lora_r": 4,
    "lora_alpha": 16,
    "lora_dropout": 0.01,
    "target_modules": ["q_proj", "v_proj"],
    "print_every_steps": 20,
    "seed": 42,
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.manual_seed(CONFIG["seed"])
random.seed(CONFIG["seed"])

# -------------------------
# 1. 加载 tokenizer（Qwen2 正确范式）
# -------------------------
print("Loading tokenizer from", CONFIG["base_model_path"])

tokenizer_path = os.path.join(CONFIG["base_model_path"], "tokenizer.json")
if os.path.exists(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_path)
else:
    tokenizer = AutoTokenizer.from_pretrained(CONFIG["base_model_path"], trust_remote_code=True)

# ======【Qwen2 推荐标准设置】======
# 1) 明确设置 eos / bos（大多数 qwen2 tokenizer.json 自带，但确保一致）
tokenizer.eos_token = "<|endoftext|>"
tokenizer.bos_token = "<|startoftext|>"

# 2) 添加 im_start / im_end
# 注意：这一步会重建 vocab → pad_token 会丢失
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<|im_start|>", "<|im_end|>"]
})

# 3) ★ 关键：add_special_tokens 后必须重新设置 pad_token
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded.")
print("  eos_token:", tokenizer.eos_token)
print("  bos_token:", tokenizer.bos_token)
print("  pad_token:", tokenizer.pad_token)
print("  pad_token_id:", tokenizer.pad_token_id)
print("  vocab_size:", len(tokenizer))
# =================================

# -------------------------
# 2. 从 Ulysses txt 生成 token chunks（滑动窗口）
# -------------------------
def load_and_tokenize_txt(txt_path, tokenizer, max_len, stride):
    print("Reading Ulysses txt:", txt_path)
    with open(txt_path, "r", encoding="utf-8") as f:
        raw = f.read()

    # 简单按段落拆分（保留语义完整性），段落间保留换行
    paragraphs = [p.strip() for p in raw.split("\n\n") if p.strip()]
    print(f"Found {len(paragraphs)} paragraphs; will pack paragraphs into chunks <= {max_len} tokens.")

    all_chunks = []
    cur_ids = []
    cur_chars = []

    for para in paragraphs:
        para_ids = tokenizer(para, add_special_tokens=False)["input_ids"]
        if len(para_ids) > max_len:
            # 段落本身超长：按 token 窗口切分
            start = 0
            while start < len(para_ids):
                end = min(start + max_len, len(para_ids))
                chunk_ids = para_ids[start:end]
                chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
                all_chunks.append(chunk_text)
                if end == len(para_ids):
                    break
                start += max_len - stride
            continue

        # 若加入当前 chunk 后不超长则合并
        if len(cur_ids) + len(para_ids) <= max_len:
            cur_ids.extend(para_ids)
            cur_chars.append(para)
        else:
            # flush current
            if cur_ids:
                all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))
            # start new
            cur_ids = para_ids.copy()
            cur_chars = [para]

    # flush tail
    if cur_ids:
        all_chunks.append(tokenizer.decode(cur_ids, skip_special_tokens=True))

    # 进一步应用滑动窗口以增加多样性（可选）
    # 这里我们再把每 chunk 做 stride 分割，避免遗漏跨段上下文
    final_chunks = []
    for chunk in all_chunks:
        ids = tokenizer(chunk, add_special_tokens=False)["input_ids"]
        start = 0
        while start < len(ids):
            end = min(start + max_len, len(ids))
            part = tokenizer.decode(ids[start:end], skip_special_tokens=True)
            final_chunks.append(part)
            if end == len(ids):
                break
            start += max_len - stride

    print(f"Total token chunks prepared: {len(final_chunks)}")
    return final_chunks

chunks = load_and_tokenize_txt(CONFIG["ulysses_txt"], tokenizer, CONFIG["max_len"], CONFIG["stride"])

# -------------------------
# 3. 构造 Dataset（对话格式 + prefix-only mask）
# -------------------------
class UlyssesDataset(Dataset):
    def __init__(self, chunks, tokenizer, max_len):
        self.examples = []
        self.tokenizer = tokenizer
        self.max_len = max_len

        for idx, chunk_text in enumerate(chunks):
            # user instruction: 要求生成意识流风格的段落（不抄袭原文）
            user_instr = (
                "Write an original paragraph in James Joyce's stream-of-consciousness style "
                "that captures similar imagery and tone as the following excerpt. "
                "Do not copy exact phrases; be original.\n\n"
                f"Excerpt:\n{chunk_text}"
            )

            # We will teach the model to generate text in this style by using the chunk_text
            # as the assistant target. (You can replace this by paraphrases if desired.)
            assistant_target = chunk_text

            full = f"<|im_start|>user\n{user_instr}<|im_end|>\n<|im_start|>assistant\n{assistant_target}<|im_end|>"
            enc = tokenizer(full, max_length=self.max_len, truncation=True, padding="max_length", return_tensors="pt")

            input_ids = enc["input_ids"].squeeze(0)
            attention_mask = enc["attention_mask"].squeeze(0)
            labels = input_ids.clone()

            # find assistant start token index to mask prefix
            # We search for the tokenizer encoding of "<|im_start|>assistant\n"
            # Simpler: find the first occurrence of "<|im_start|>" after the user block.
            special_id = tokenizer.convert_tokens_to_ids("<|im_start|>")
            # locate positions
            ids_list = input_ids.tolist()
            assistant_pos = 0
            try:
                # find second occurrence of im_start (first is user)
                first = ids_list.index(special_id)
                second = ids_list.index(special_id, first + 1)
                assistant_pos = second
            except ValueError:
                # fallback: try to find the textual marker
                txt = tokenizer.decode(input_ids, skip_special_tokens=False)
                marker = "<|im_start|>assistant\n"
                if marker in txt:
                    assistant_pos = tokenizer(marker, add_special_tokens=False)["input_ids"][0]
                    # not reliable; fallback to 0
                else:
                    assistant_pos = 0

            # mask prefix (user + instruction) so loss is only computed on assistant part
            labels[:assistant_pos] = -100

            self.examples.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": labels
            })

        print(f"Dataset built: {len(self.examples)} examples")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

dataset = UlyssesDataset(chunks, tokenizer, CONFIG["max_len"])
dataloader = DataLoader(dataset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=0, drop_last=True)

# -------------------------
# 4. 加载基座模型并装上 LoRA
# -------------------------
print("Loading base model from:", CONFIG["base_model_path"])
model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model_path"],
    torch_dtype=torch.float16 if CONFIG["fp16"] else torch.float32,
    device_map="auto",
    low_cpu_mem_usage=True
)

lora_cfg = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=CONFIG["target_modules"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()
model.train()

# device for inputs: model may be sharded; use next(model.parameters()).device
model_device = next(model.parameters()).device
print("Model device:", model_device)

# -------------------------
# 5. optimizer & loss
# -------------------------
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"])
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

# -------------------------
# 6. 训练循环
# -------------------------
def format_time(seconds):
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

total_steps = len(dataloader) * CONFIG["num_train_epochs"] // CONFIG["gradient_accumulation_steps"]
print("Training config -- epochs:", CONFIG["num_train_epochs"], "batches_per_epoch:", len(dataloader), "total_steps(approx):", total_steps)

global_step = 0
for epoch in range(CONFIG["num_train_epochs"]):
    epoch_start = time.time()
    running_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(dataloader):
        global_step += 1

        input_ids = batch["input_ids"].to(model_device)
        attention_mask = batch["attention_mask"].to(model_device)
        labels = batch["labels"].to(model_device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / CONFIG["gradient_accumulation_steps"]
        loss.backward()

        if global_step % CONFIG["gradient_accumulation_steps"] == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.5)
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item() * CONFIG["gradient_accumulation_steps"]

        if global_step % CONFIG["print_every_steps"] == 0:
            avg_loss = running_loss / (global_step if global_step>0 else 1)
            print(f"[Epoch {epoch+1}] Step {global_step} | loss: {avg_loss:.4f} | step_time: {format_time(time.time()-epoch_start)}")

    epoch_time = format_time(time.time() - epoch_start)
    print(f"Epoch {epoch+1} finished in {epoch_time}")

# -------------------------
# 7. 保存微调后模型（包含 LoRA）
# -------------------------
print("Saving finetuned model to:", CONFIG["output_dir"])
model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print("Done.")


Loading tokenizer from /root/autodl-tmp/Pro/qwen2.5-sft-full
Tokenizer loaded.
  eos_token: <|endoftext|>
  bos_token: <|startoftext|>
  pad_token: <|endoftext|>
  pad_token_id: 151643
  vocab_size: 151665
Reading Ulysses txt: /root/autodl-tmp/Pro/Ulysses尤利西斯.txt
Found 7169 paragraphs; will pack paragraphs into chunks <= 1024 tokens.
Total token chunks prepared: 459


`torch_dtype` is deprecated! Use `dtype` instead!


Dataset built: 459 examples
Loading base model from: /root/autodl-tmp/Pro/qwen2.5-sft-full


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.12it/s]
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


trainable params: 1,261,568 || all params: 7,616,878,080 || trainable%: 0.0166
Model device: cuda:0
Training config -- epochs: 2 batches_per_epoch: 459 total_steps(approx): 229
[Epoch 1] Step 20 | loss: 2.4292 | step_time: 00:00:04
[Epoch 1] Step 40 | loss: 2.8493 | step_time: 00:00:07
[Epoch 1] Step 60 | loss: 3.1239 | step_time: 00:00:11
[Epoch 1] Step 80 | loss: 2.8827 | step_time: 00:00:15
[Epoch 1] Step 100 | loss: 2.8391 | step_time: 00:00:19
[Epoch 1] Step 120 | loss: 2.8489 | step_time: 00:00:22
[Epoch 1] Step 140 | loss: 2.8245 | step_time: 00:00:26
[Epoch 1] Step 160 | loss: 2.8936 | step_time: 00:00:30
[Epoch 1] Step 180 | loss: 2.7989 | step_time: 00:00:34
[Epoch 1] Step 200 | loss: 2.7810 | step_time: 00:00:37
[Epoch 1] Step 220 | loss: 2.7274 | step_time: 00:00:41
[Epoch 1] Step 240 | loss: 2.6937 | step_time: 00:00:45
[Epoch 1] Step 260 | loss: 2.6742 | step_time: 00:00:48
[Epoch 1] Step 280 | loss: 2.6603 | step_time: 00:00:52
[Epoch 1] Step 300 | loss: 2.6799 | step_ti

In [4]:
# inference_mindflow.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置路径
# -------------------------
BASE_MODEL = "/root/autodl-tmp/Pro/qwen2.5-sft-full"               # 基座（模型一）
LORA_MODEL = "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"      # 尤利西斯心流微调（模型二）
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 200,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.12,
}

# -------------------------
# 2. 加载 tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# 必须设置 pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. 加载模型 + LoRA
# -------------------------
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, LORA_MODEL)

model.eval()

# -------------------------
# 4. 推理函数
# -------------------------
def generate_with_mindflow(prompt: str):
    # 你训练时使用的对话格式
    formatted = (
        f"<|im_start|>user\n{prompt}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )

    input_ids = tokenizer(
        formatted,
        return_tensors="pt"
    ).input_ids.to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **GEN_CFG
        )

    text = tokenizer.decode(output[0], skip_special_tokens=False)

    # 清理掉格式 token
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    return text


# -------------------------
# 5. 测试
# -------------------------
if __name__ == "__main__":
    test_prompt = (
        "write a poem with keyword: willow, wind, fluffy, charmed. "
        "With a mindflow of thoughts."
    )

    print("\n=== Generated Poem ===\n")
    print(generate_with_mindflow(test_prompt))


Loading tokenizer...
Loading base model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.18it/s]
/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Loading LoRA adapter...

=== Generated Poem ===

In the heart of an old, forgotten glade,
Where shadows dance and whispers play,
Stands a willow, mighty and grand,
With branches swaying to the land.

The breeze that stirs beneath her boughs
Is not just any gusty flow;
It's wind itself, enchanted, turned,
To softly caress what it has earned.

Her leaves rustle in a lullaby,
A symphony where fluffiness lies.
Fluffy down from fields afar,
Drifts into the air like a fairy car.

She weaves them into dreams so pure,
Charmed by the gentle touch of the lure.
Each strand entwines as if she knows
Of magic hidden, unseen and aglow.

Thus, under her spell, the world bends,
A tapestry of light and shades, friends.
For in this place where willows sway,
Life breathes easy, every day.


In [1]:
import torch
import time
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置
# -------------------------
BASE_MODEL = "/root/autodl-tmp/Pro/qwen2.5-sft-full"
LORA_MODEL = "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 400,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.12,
}

# 新增：保存配置
SAVE_FOLDER = "./poetry_results"  # 保存目录
os.makedirs(SAVE_FOLDER, exist_ok=True)  # 自动创建目录（不存在时）

# -------------------------
# 2. 加载 tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 3. 加载模型 + LoRA
# -------------------------
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Loading LoRA adapter...")
model = PeftModel.from_pretrained(model, LORA_MODEL)
model.eval()

# -------------------------
# 4. 核心：带“记忆”的生成
# -------------------------
conversation_memory = ""   # ⭐ 存上一轮生成结果


def generate_with_memory(user_prompt: str):
    global conversation_memory

    if conversation_memory:
        # 在已有诗歌基础上 refinement
        formatted = (
            "<|im_start|>user\n"
            "Here is the previous poem:\n"
            f"{conversation_memory}\n\n"
            "Please revise or continue it according to the following instruction:\n"
            f"{user_prompt}"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
    else:
        # 第一轮
        formatted = (
            f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )

    input_ids = tokenizer(formatted, return_tensors="pt").input_ids.to(DEVICE)

    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **GEN_CFG
        )

    text = tokenizer.decode(output[0], skip_special_tokens=False)

    # 清理输出
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    # ⭐ 更新记忆
    conversation_memory = text

    return text


# -------------------------
# 5. 交互式输入
# -------------------------
if __name__ == "__main__":
    print("\n🧠 Mindflow Poetry Generator")
    print("输入 prompt 回车生成")
    print("输入 reset 清空记忆")
    print("输入 exit 退出并保存终版诗歌\n")

    while True:
        user_input = input(">>> ").strip()

        if user_input.lower() == "exit":
            # 保存终版诗歌到txt文件
            if conversation_memory:
                # 生成带时间戳的文件名，避免覆盖
                timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
                file_path = os.path.join(SAVE_FOLDER, f"final_poem_{timestamp}.txt")
                
                # 写入文件（UTF-8编码防止中文乱码）
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write("=== Mindflow Poetry Final Version ===\n")
                    f.write(f"Generated Time: {time.ctime()}\n")
                    f.write("=" * 40 + "\n\n")
                    f.write(conversation_memory)
                
                print(f"\n✅ 终版诗歌已保存至：{file_path}")
            else:
                print("\n⚠️  暂无生成的诗歌内容，未保存文件")
            
            print("Bye.")
            break

        if user_input.lower() == "reset":
            conversation_memory = ""
            print("🔄 Memory cleared.\n")
            continue

        output = generate_with_memory(user_input)

        print("\n=== Generated ===\n")
        print(output)
        print("\n=================\n")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading base model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


...


=== Generated ===

Oh Nature, vast and wondrous,
In fields where grasses sigh and sway,
A symphony of chirps and songs,
From dawn's first light till night's last ray.

The rooster crows upon my farm,
His voice rings out with hearty cheer,
And larks ascend to heavens' arm,
Their melodies sweet, clear, and near.

Prairie chickens strut and dance,
As goldenrod and daisies bloom,
While cornfields stretch to horizons chance,
And bluebirds dart through open rooms.

My heart swells wide as boundless skies,
With joy that springs from every nook,
For here among these verdant prizes,
I find contentment, free from clutch.

So let me wander on and on,
Through woodlands deep and sunny glades,
Where foxes run and wildflowers hon,
Nature whispers truths untamed.

This land, this life, is all I need—
No other wealth could e'er compare.
Chirping thrushes, singing streams fed,
Teach wisdom worth remembering.
.Disclaimer: This response is generated text designed to emulate the style of Walt Whitman but 

>>>  delete the disclaimer.



=== Generated ===

Oh Nature, vast and wondrous,
In fields where grasses sigh and sway,
A symphony of chirps and songs,
From dawn's first light till night's last ray.

The rooster crows upon my farm,
His voice rings out with hearty cheer,
And larks ascend to heavens' arm,
Their melodies sweet, clear, and near.

Prairie chickens strut and dance,
As goldenrod and daisies bloom,
While cornfields stretch to horizons chance,
And bluebirds dart through open rooms.

My heart swells wide as boundless skies,
With joy that springs from every nook,
For here among these verdant prizes,
I find contentment, free from clutch.

So let me wander on and on,
Through woodlands deep and sunny glades,
Where foxes run and wildflowers hon,
Nature whispers truths untamed.

This land, this life, is all I need—
No other wealth could e'er compare.
Chirping thrushes, singing streams fed,
Teach wisdom worth remembering.




>>>  exit



✅ 终版诗歌已保存至：./poetry_results/final_poem_20251213_172423.txt
Bye.


In [1]:
import torch
import time
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# -------------------------
# 1. 配置
# -------------------------
# 模型路径配置
MODEL_PATHS = {
    "base": "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b",
    "sft": "/root/autodl-tmp/Pro/qwen2.5-sft-full",
    "mindflow": {
        "base": "/root/autodl-tmp/Pro/qwen2.5-sft-full",
        "lora": "/root/autodl-tmp/Pro/qwen2.5-mindflow-ulysses"
    }
}

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

GEN_CFG = {
    "max_new_tokens": 400,
    "temperature": 0.85,
    "top_k": 40,
    "top_p": 0.92,
    "repetition_penalty": 1.12,
}

# 新增：保存配置
SAVE_FOLDER = "./poetry_results"  # 保存目录
os.makedirs(SAVE_FOLDER, exist_ok=True)  # 自动创建目录（不存在时）

# -------------------------
# 2. 全局变量：多模型记忆存储
# -------------------------
# 分别存储三个模型的对话记忆
conversation_memory = {
    "base": "",
    "sft": "",
    "mindflow": ""
}
tokenizer = None  # 全局tokenizer
models = {}  # 存储加载的模型

# -------------------------
# 3. 加载 tokenizer
# -------------------------
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATHS["base"],
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# -------------------------
# 4. 加载所有模型
# -------------------------
def load_all_models():
    """加载所有三个模型"""
    # 加载Base模型
    print("\nLoading BASE model...")
    models["base"] = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["base"],
        torch_dtype=torch.float16,
        device_map="auto"
    ).eval()

    # 加载SFT模型
    print("Loading SFT model...")
    models["sft"] = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["sft"],
        torch_dtype=torch.float16,
        device_map="auto"
    ).eval()

    # 加载Mindflow模型（base + LoRA）
    print("Loading MINDFLOW model (base + LoRA)...")
    mindflow_base = AutoModelForCausalLM.from_pretrained(
        MODEL_PATHS["mindflow"]["base"],
        torch_dtype=torch.float16,
        device_map="auto"
    )
    models["mindflow"] = PeftModel.from_pretrained(
        mindflow_base,
        MODEL_PATHS["mindflow"]["lora"]
    ).eval()

    print("\n✅ All models loaded successfully!")
    return models

# -------------------------
# 5. 核心：带“记忆”的多模型生成
# -------------------------
def generate_with_memory(user_prompt: str, model_type: str):
    """带记忆的生成函数（指定模型）"""
    global conversation_memory, models, tokenizer

    model = models[model_type]
    memory = conversation_memory[model_type]

    # 构建带记忆的prompt
    if memory:
        formatted = (
            "<|im_start|>user\n"
            "Here is the previous poem:\n"
            f"{memory}\n\n"
            "Please revise or continue it according to the following instruction:\n"
            f"{user_prompt}"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
        )
    else:
        # 第一轮生成
        formatted = (
            f"<|im_start|>user\n{user_prompt}<|im_end|>\n"
            f"<|im_start|>assistant\n"
        )

    # 编码输入
    input_ids = tokenizer(formatted, return_tensors="pt").input_ids.to(DEVICE)

    # 生成输出
    with torch.no_grad():
        output = model.generate(
            input_ids=input_ids,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            **GEN_CFG
        )

    # 解码并清理输出
    text = tokenizer.decode(output[0], skip_special_tokens=False)
    if "<|im_start|>assistant\n" in text:
        text = text.split("<|im_start|>assistant\n")[1]
    text = text.replace("<|im_end|>", "").replace("<|endoftext|>", "").strip()

    # 更新对应模型的记忆
    conversation_memory[model_type] = text

    return text

def generate_all_models(user_prompt: str):
    """同时调用三个模型生成结果"""
    results = {}
    
    print("\n" + "-"*80)
    # Base模型生成
    print("🔄 Generating with BASE model...")
    results["base"] = generate_with_memory(user_prompt, "base")
    
    # SFT模型生成
    print("🔄 Generating with SFT model...")
    results["sft"] = generate_with_memory(user_prompt, "sft")
    
    # Mindflow模型生成
    print("🔄 Generating with MINDFLOW model...")
    results["mindflow"] = generate_with_memory(user_prompt, "mindflow")
    print("-"*80 + "\n")
    
    return results

# -------------------------
# 6. 保存所有模型的终版结果
# -------------------------
def save_all_results():
    """保存三个模型的终版输出"""
    timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    file_path = os.path.join(SAVE_FOLDER, f"all_models_poem_{timestamp}.txt")
    
    with open(file_path, "w", encoding="utf-8") as f:
        # 写入元信息
        f.write("=== Mindflow Poetry - All Models Final Results ===\n")
        f.write(f"Generated Time: {time.ctime()}\n")
        f.write("=" * 60 + "\n\n")
        
        # 写入Base模型结果
        f.write("📌 BASE MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["base"] if conversation_memory["base"] else "No content generated\n")
        f.write("\n\n")
        
        # 写入SFT模型结果
        f.write("📌 SFT MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["sft"] if conversation_memory["sft"] else "No content generated\n")
        f.write("\n\n")
        
        # 写入Mindflow模型结果
        f.write("📌 MINDFLOW MODEL OUTPUT:\n")
        f.write("-" * 40 + "\n")
        f.write(conversation_memory["mindflow"] if conversation_memory["mindflow"] else "No content generated\n")
    
    return file_path

# -------------------------
# 7. 交互式输入
# -------------------------
if __name__ == "__main__":
    # 加载所有模型
    models = load_all_models()
    
    # 交互式界面
    print("\n\n🧠 Mindflow Poetry Generator - Multi-Model Mode")
    print("=" * 70)
    print("指令说明：")
    print("  1. 直接输入文本      - 同时调用BASE/SFT/MINDFLOW三个模型生成诗歌")
    print("  2. reset             - 清空所有模型的对话记忆")
    print("  3. exit              - 退出并保存所有模型的终版诗歌")
    print("=" * 70 + "\n")

    while True:
        user_input = input(">>> ").strip()

        if not user_input:
            print("⚠️  请输入有效内容！")
            continue

        # 退出并保存所有结果
        if user_input.lower() == "exit":
            # 检查是否有生成内容
            has_content = any(bool(v) for v in conversation_memory.values())
            if has_content:
                file_path = save_all_results()
                print(f"\n✅ 所有模型的终版诗歌已保存至：{file_path}")
            else:
                print("\n⚠️  暂无生成的诗歌内容，未保存文件")
            
            print("\n👋 Bye!")
            break

        # 清空所有记忆
        elif user_input.lower() == "reset":
            conversation_memory = {
                "base": "",
                "sft": "",
                "mindflow": ""
            }
            print("✅ 已清空所有模型的对话记忆！\n")
            continue

        # 正常生成：同时调用三个模型
        else:
            # 生成所有模型的结果
            results = generate_all_models(user_input)
            
            # 输出结果
            print("📝 BASE MODEL OUTPUT:\n")
            print(results["base"])
            print("\n" + "="*80)
            
            print("📝 SFT MODEL OUTPUT:\n")
            print(results["sft"])
            print("\n" + "="*80)
            
            print("📝 MINDFLOW MODEL OUTPUT:\n")
            print(results["mindflow"])
            print("\n" + "="*80 + "\n")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Loading tokenizer...

Loading BASE model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]


Loading SFT model...


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]


...

>>>  write a poem in Walt whitman style, using the keyword: nature, chirping, farm


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



--------------------------------------------------------------------------------
🔄 Generating with BASE model...
🔄 Generating with SFT model...
🔄 Generating with MINDFLOW model...
--------------------------------------------------------------------------------

📝 BASE MODEL OUTPUT:

In fields where the sun’s warm hands caress,
And nature whispers to the whispering breeze,
I hear the songs that fill this vast expanse—
Chirping crickets and larks ascending.

Amidst rows of corn and wheat so green,
Where cows graze tranquil as their breaths are seen,
A farmer tills with slow, sure pace and might,
His heart with every furrow taking flight.

Beneath the big blue sky's expansive dome,
He works the earth, the very essence of home.
The soil rich with the promise of tomorrow,
As seasons turn like leaves upon the floor.

This land, so wild yet cultivated too,
Sings with life in voices low and true.
The chirping thrums through the air, light as thought,
Nature’s symphony, on and around the plot.

>>>  exit



✅ 所有模型的终版诗歌已保存至：./poetry_results/all_models_poem_20251213_180850.txt

👋 Bye!


In [3]:
import torch
import json
import os
from typing import Dict, Optional
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# -------------------------
# 1. 配置本地 Qwen2.5-7B 路径 & 文件路径
# -------------------------
# 本地 Qwen2.5-7B 模型权重路径（替换为你的实际路径）
LOCAL_QWEN_PATH = "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b"  # 示例路径，需修改
# 诗歌文件路径
POEM_FILE_PATH = "/root/autodl-tmp/Pro/poetry_results/final_poem_20251213_172423.txt"

# 生成配置（适配 7B 模型，平衡速度和效果）
GEN_CONFIG = GenerationConfig(
    temperature=0.0,  # 固定温度保证评分稳定
    top_k=40,
    top_p=0.92,
    repetition_penalty=1.1,
    max_new_tokens=1500,  # 足够容纳评分+分析
    eos_token_id=151643,  # Qwen 的 eos_token_id
    pad_token_id=151643,  # Qwen 的 pad_token_id
    device_map="auto",    # 自动分配显存（GPU/CPU）
)

# -------------------------
# 2. 加载本地 Qwen2.5-7B 模型 & Tokenizer
# -------------------------
print("🔄 正在加载本地 Qwen2.5-7B 模型...")
tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_QWEN_PATH,
    trust_remote_code=True,
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    LOCAL_QWEN_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16,  # 用 float16 节省显存
    device_map="auto",
    load_in_4bit=True,  # 可选：4bit 量化，进一步降低显存占用（需安装 bitsandbytes）
)
model.eval()
print("✅ Qwen2.5-7B 模型加载完成！")

# -------------------------
# 3. 读取文本文件中的诗歌（逻辑不变）
# -------------------------
def load_poem_from_file(file_path: str) -> str:
    """读取指定路径的诗歌文本，清理无关内容"""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Poem file not found: {file_path}")
    
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    
    # 清理文本（过滤标题/注释行，仅保留诗歌正文）
    poem_lines = []
    for line in lines:
        line = line.strip()
        # 过滤LitBench生成的注释行（如===开头、Generated Time等）
        if not line or line.startswith("===") or "Generated Time" in line:
            continue
        poem_lines.append(line)
    
    poem_text = "\n".join(poem_lines)
    if not poem_text:
        raise ValueError("No valid poem content found in the file")
    
    return poem_text

# -------------------------
# 4. 构建LitBench评分+详细分析的Prompt（适配Qwen）
# -------------------------
def build_analysis_prompt(poem_text: str, theme: Optional[str] = None) -> str:
    """
    构建Prompt：适配Qwen格式，保留LitBench评分维度
    """
    theme = theme if theme else "no specified theme"
    prompt = f"""
    You are a professional English poetry critic following the LitBench benchmark for creative writing evaluation.
    Evaluate the following English poem with TWO parts, and output ONLY JSON format (no extra text, no explanation):
    
    ## PART 1: Scoring (0-10 scale, 10 = highest)
    Score the poem on these dimensions:
    1. Fluency: Line/rhythm smoothness, no semantic fragmentation/awkward phrasing.
    2. Emotional Resonance: Ability to evoke genuine reader emotion (joy/sorrow/nostalgia etc.).
    3. Image Innovation: Originality of imagery/rhetoric (metaphor/simile/personification) – avoid clichés.
    4. Theme Consistency: Alignment with the given theme ({theme}) – no irrelevant content.
    5. Human Preference: Fit with human poetic aesthetic (avoid AI-generated templated language).
    
    ## PART 2: Detailed Analysis
    For each dimension, write a 50-100 word analysis explaining the score:
    - For Fluency: Point out specific lines with good/bad rhythm, or semantic issues.
    - For Emotional Resonance: Explain which emotions are evoked and how (word choice/tone).
    - For Image Innovation: Identify specific imagery/rhetoric, comment on originality (clichés if any).
    - For Theme Consistency: Link specific lines to the theme (or note deviations).
    - For Human Preference: Comment on whether the language feels human-written vs AI-templated.
    
    ## Poem Text
    {poem_text}
    
    ## Output Format (JSON ONLY, no extra text)
    {{
        "scores": {{
            "fluency": 8.5,
            "emotional_resonance": 9.0,
            "image_innovation": 7.5,
            "theme_consistency": 8.0,
            "human_preference": 8.8,
            "total_score": 8.36
        }},
        "analysis": {{
            "fluency": "Detailed analysis for fluency...",
            "emotional_resonance": "Detailed analysis for emotional resonance...",
            "image_innovation": "Detailed analysis for image innovation...",
            "theme_consistency": "Detailed analysis for theme consistency...",
            "human_preference": "Detailed analysis for human preference..."
        }}
    }}
    
    Calculate total_score as the average of the 5 dimensions (round to 2 decimal places).
    Keep analysis concise (50-100 words per dimension) and specific (reference poem lines).
    """
    return prompt.strip()

# -------------------------
# 5. 调用本地 Qwen2.5-7B 分析诗歌（替代GPT-4）
# -------------------------
def analyze_english_poem(poem_text: str, theme: Optional[str] = None) -> Dict:
    """
    调用本地 Qwen2.5-7B：输出LitBench评分 + 各维度详细分析
    """
    prompt = build_analysis_prompt(poem_text, theme)
    
    try:
        # 构建 Qwen 格式的对话
        messages = [
            {"role": "user", "content": prompt}
        ]
        # 格式化 prompt（Qwen 要求的格式）
        input_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        # 编码输入
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        
        # 生成结果
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                generation_config=GEN_CONFIG
            )
        
        # 解码输出
        response = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)
        result_str = response.strip()
        
        # 解析 JSON 结果
        result_dict = json.loads(result_str)
        
        # 手动校准总分（防止模型计算错误）
        dimensions = ["fluency", "emotional_resonance", "image_innovation", "theme_consistency", "human_preference"]
        calculated_total = round(sum(result_dict["scores"][d] for d in dimensions) / 5, 2)
        result_dict["scores"]["total_score"] = calculated_total
        
        return result_dict
    
    except json.JSONDecodeError:
        return {"error": f"Failed to parse Qwen response (invalid JSON). Raw response: {result_str}"}
    except Exception as e:
        return {"error": f"Qwen generation failed: {str(e)}"}

# -------------------------
# 6. 格式化输出结果（逻辑不变）
# -------------------------
def print_analysis_result(result: Dict):
    """美观输出评分和分析结果"""
    if "error" in result:
        print(f"❌ Analysis Failed: {result['error']}")
        return
    
    print("="*80)
    print("📜 Poem Content:")
    print(load_poem_from_file(POEM_FILE_PATH))
    print("="*80)
    
    print("\n🎯 LitBench Score Summary (0-10):")
    scores = result["scores"]
    for dim, score in scores.items():
        print(f"  - {dim.replace('_', ' ').title()}: {score}")
    
    print("\n📝 Detailed Dimension Analysis:")
    analysis = result["analysis"]
    for dim, text in analysis.items():
        print(f"\n【{dim.replace('_', ' ').title()}】")
        print(f"  {text}")
    print("="*80)

# -------------------------
# 7. 主函数：读取诗歌 → 分析 → 输出
# -------------------------
if __name__ == "__main__":
    try:
        # 1. 读取诗歌文件
        poem = load_poem_from_file(POEM_FILE_PATH)
        
        # 2. 调用本地 Qwen2.5-7B 分析
        print("🚀 开始调用本地 Qwen2.5-7B 分析诗歌...")
        analysis_result = analyze_english_poem(poem, theme="love and alienation")
        
        # 3. 输出结果
        print_analysis_result(analysis_result)
        
        # 可选：保存分析结果到文件
        with open("poem_analysis_result_qwen.json", "w", encoding="utf-8") as f:
            json.dump(analysis_result, f, indent=2)
        print("\n✅ Analysis result saved to: poem_analysis_result_qwen.json")
    
    except FileNotFoundError as e:
        print(f"❌ {e}")
    except ValueError as e:
        print(f"❌ {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")

/root/miniconda3/envs/qwen_text/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
`torch_dtype` is deprecated! Use `dtype` instead!


🔄 正在加载本地 Qwen2.5-7B 模型...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.26it/s]


✅ Qwen2.5-7B 模型加载完成！
🚀 开始调用本地 Qwen2.5-7B 分析诗歌...
📜 Poem Content:
Oh Nature, vast and wondrous,
In fields where grasses sigh and sway,
A symphony of chirps and songs,
From dawn's first light till night's last ray.
The rooster crows upon my farm,
His voice rings out with hearty cheer,
And larks ascend to heavens' arm,
Their melodies sweet, clear, and near.
Prairie chickens strut and dance,
As goldenrod and daisies bloom,
While cornfields stretch to horizons chance,
And bluebirds dart through open rooms.
My heart swells wide as boundless skies,
With joy that springs from every nook,
For here among these verdant prizes,
I find contentment, free from clutch.
So let me wander on and on,
Through woodlands deep and sunny glades,
Where foxes run and wildflowers hon,
Nature whispers truths untamed.
This land, this life, is all I need—
No other wealth could e'er compare.
Chirping thrushes, singing streams fed,
Teach wisdom worth remembering.

🎯 LitBench Score Summary (0-10):
  - Fluency: 8.5
  - 

In [2]:
import torch
import json
import os
from typing import Dict, Optional, List
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# -------------------------
# 1. 配置本地 Qwen2.5-7B 路径 & 文件路径
# -------------------------
# 本地 Qwen2.5-7B 模型权重路径（替换为你的实际路径）
LOCAL_QWEN_PATH = "/root/autodl-tmp/qwen2.5-7b/qwen2.5-7b"  # 示例路径，需修改
# 包含三个模型诗歌的文件路径（多模型生成的结果文件）
POEM_FILE_PATH = "/root/autodl-tmp/Pro/poetry_results/all_models_poem_20251213_180850.txt"

# 生成配置（适配 7B 模型，平衡速度和效果）
GEN_CONFIG = GenerationConfig(
    temperature=0.0,  # 固定温度保证评分稳定
    top_k=40,
    top_p=0.92,
    repetition_penalty=1.1,
    max_new_tokens=1500,  # 足够容纳评分+分析
    eos_token_id=151643,  # Qwen 的 eos_token_id
    pad_token_id=151643,  # Qwen 的 pad_token_id
    device_map="auto",    # 自动分配显存（GPU/CPU）
)

# -------------------------
# 2. 加载本地 Qwen2.5-7B 模型 & Tokenizer
# -------------------------
print("🔄 正在加载本地 Qwen2.5-7B 模型...")
tokenizer = AutoTokenizer.from_pretrained(
    LOCAL_QWEN_PATH,
    trust_remote_code=True,
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    LOCAL_QWEN_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16,  # 用 float16 节省显存
    device_map="auto",
    load_in_4bit=True,  # 可选：4bit 量化，进一步降低显存占用（需安装 bitsandbytes）
)
model.eval()
print("✅ Qwen2.5-7B 模型加载完成！")

# -------------------------
# 3. 解析多模型诗歌文件（新增核心逻辑）
# -------------------------
def parse_multi_model_poem_file(file_path: str) -> Dict[str, str]:
    """
    解析包含三个模型诗歌的文件，提取每个模型的诗歌内容
    适配格式：
    📌 BASE MODEL OUTPUT:
    ------------------------
    诗歌内容...
    
    📌 SFT MODEL OUTPUT:
    ------------------------
    诗歌内容...
    
    📌 MINDFLOW MODEL OUTPUT:
    ------------------------
    诗歌内容...
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Poem file not found: {file_path}")
    
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()
    
    # 定义模型标记和对应的key
    model_markers = {
        "📌 BASE MODEL OUTPUT:": "base",
        "📌 SFT MODEL OUTPUT:": "sft",
        "📌 MINDFLOW MODEL OUTPUT:": "mindflow"
    }
    
    # 初始化结果
    poem_dict = {
        "base": "",
        "sft": "",
        "mindflow": ""
    }
    
    # 分割内容并提取每个模型的诗歌
    content_parts = content.split("📌")
    for part in content_parts:
        if not part.strip():
            continue
        
        # 匹配模型类型
        for marker, model_key in model_markers.items():
            if marker.replace("📌 ", "") in part:
                # 提取诗歌内容（过滤标记、分隔线）
                lines = part.split("\n")
                poem_lines = []
                in_poem = False
                
                for line in lines:
                    line = line.strip()
                    # 跳过标记行和分隔线
                    if line == marker.replace("📌 ", "") or line.startswith("---"):
                        in_poem = True
                        continue
                    # 跳过空行（但保留诗歌内的空行）
                    if in_poem and line:
                        poem_lines.append(line)
                    # 遇到下一个模型标记则停止
                    if any(marker.replace("📌 ", "") in line for marker in model_markers.keys()):
                        break
                
                poem_dict[model_key] = "\n".join(poem_lines)
                break
    
    # 检查是否提取到有效内容
    empty_models = [k for k, v in poem_dict.items() if not v.strip()]
    if empty_models:
        raise ValueError(f"Failed to extract poem content for models: {', '.join(empty_models)}")
    
    return poem_dict

# -------------------------
# 4. 构建LitBench评分+详细分析的Prompt（适配Qwen）
# -------------------------
def build_analysis_prompt(poem_text: str, model_type: str, theme: Optional[str] = None) -> str:
    """
    构建Prompt：适配Qwen格式，保留LitBench评分维度，标注模型类型
    """
    theme = theme if theme else "no specified theme"
    prompt = f"""
    You are a professional English poetry critic following the LitBench benchmark for creative writing evaluation.
    Evaluate the following English poem generated by {model_type.upper()} model with TWO parts, and output ONLY JSON format (no extra text, no explanation):
    
    ## PART 1: Scoring (0-10 scale, 10 = highest)
    Score the poem on these dimensions:
    1. Fluency: Line/rhythm smoothness, no semantic fragmentation/awkward phrasing.
    2. Emotional Resonance: Ability to evoke genuine reader emotion (joy/sorrow/nostalgia etc.).
    3. Image Innovation: Originality of imagery/rhetoric (metaphor/simile/personification) – avoid clichés.
    4. Theme Consistency: Alignment with the given theme ({theme}) – no irrelevant content.
    5. Human Preference: Fit with human poetic aesthetic (avoid AI-generated templated language).
    
    ## PART 2: Detailed Analysis
    For each dimension, write a 50-100 word analysis explaining the score:
    - For Fluency: Point out specific lines with good/bad rhythm, or semantic issues.
    - For Emotional Resonance: Explain which emotions are evoked and how (word choice/tone).
    - For Image Innovation: Identify specific imagery/rhetoric, comment on originality (clichés if any).
    - For Theme Consistency: Link specific lines to the theme (or note deviations).
    - For Human Preference: Comment on whether the language feels human-written vs AI-templated.
    
    ## Poem Text (from {model_type.upper()} model)
    {poem_text}
    
    ## Output Format (JSON ONLY, no extra text)
    {{
        "model_type": "{model_type}",
        "scores": {{
            "fluency": 8.5,
            "emotional_resonance": 9.0,
            "image_innovation": 7.5,
            "theme_consistency": 8.0,
            "human_preference": 8.8,
            "total_score": 8.36
        }},
        "analysis": {{
            "fluency": "Detailed analysis for fluency...",
            "emotional_resonance": "Detailed analysis for emotional resonance...",
            "image_innovation": "Detailed analysis for image innovation...",
            "theme_consistency": "Detailed analysis for theme consistency...",
            "human_preference": "Detailed analysis for human preference..."
        }}
    }}
    
    Calculate total_score as the average of the 5 dimensions (round to 2 decimal places).
    Keep analysis concise (50-100 words per dimension) and specific (reference poem lines).
    """
    return prompt.strip()

# -------------------------
# 5. 调用本地 Qwen2.5-7B 分析单首诗歌
# -------------------------
def analyze_single_poem(poem_text: str, model_type: str, theme: Optional[str] = None) -> Dict:
    """
    调用本地 Qwen2.5-7B：分析单个模型生成的诗歌
    """
    prompt = build_analysis_prompt(poem_text, model_type, theme)
    
    try:
        # 构建 Qwen 格式的对话
        messages = [
            {"role": "user", "content": prompt}
        ]
        # 格式化 prompt（Qwen 要求的格式）
        input_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        # 编码输入
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
        
        # 生成结果
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                generation_config=GEN_CONFIG
            )
        
        # 解码输出
        response = tokenizer.decode(outputs[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)
        result_str = response.strip()
        
        # 解析 JSON 结果
        result_dict = json.loads(result_str)
        
        # 手动校准总分（防止模型计算错误）
        dimensions = ["fluency", "emotional_resonance", "image_innovation", "theme_consistency", "human_preference"]
        calculated_total = round(sum(result_dict["scores"][d] for d in dimensions) / 5, 2)
        result_dict["scores"]["total_score"] = calculated_total
        
        return result_dict
    
    except json.JSONDecodeError:
        return {
            "model_type": model_type,
            "error": f"Failed to parse Qwen response (invalid JSON). Raw response: {result_str}"
        }
    except Exception as e:
        return {
            "model_type": model_type,
            "error": f"Qwen generation failed: {str(e)}"
        }

# -------------------------
# 6. 批量分析所有模型的诗歌（新增核心逻辑）
# -------------------------
def batch_analyze_poems(poem_dict: Dict[str, str], theme: Optional[str] = None) -> Dict[str, Dict]:
    """
    批量分析三个模型的诗歌
    """
    analysis_results = {}
    
    # 按顺序分析每个模型的诗歌
    model_order = ["base", "sft", "mindflow"]
    for model_type in model_order:
        print(f"\n🚀 开始分析 {model_type.upper()} 模型生成的诗歌...")
        poem_text = poem_dict[model_type]
        result = analyze_single_poem(poem_text, model_type, theme)
        analysis_results[model_type] = result
        print(f"✅ {model_type.upper()} 模型诗歌分析完成！")
    
    return analysis_results

# -------------------------
# 7. 格式化输出所有分析结果
# -------------------------
def print_batch_analysis_result(analysis_results: Dict[str, Dict], poem_dict: Dict[str, str]):
    """美观输出所有模型的评分和分析结果"""
    print("\n" + "="*100)
    print("📊 多模型诗歌 LitBench 评分对比分析")
    print("="*100)
    
    # 先输出总分对比
    print("\n🎯 各模型总分对比 (0-10):")
    print("-"*60)
    for model_type, result in analysis_results.items():
        if "error" in result:
            score = "Error"
        else:
            score = result["scores"]["total_score"]
        print(f"  • {model_type.upper()}: {score}")
    
    # 逐个输出详细分析
    for model_type, result in analysis_results.items():
        print("\n" + "="*100)
        print(f"📜 {model_type.upper()} MODEL POEM & ANALYSIS")
        print("="*100)
        
        # 输出诗歌内容
        print("\n🖋️ Poem Content:")
        print("-"*40)
        print(poem_dict[model_type])
        
        # 输出分析结果
        if "error" in result:
            print(f"\n❌ Analysis Failed: {result['error']}")
            continue
        
        print("\n🎯 LitBench Score Summary (0-10):")
        print("-"*40)
        scores = result["scores"]
        for dim, score in scores.items():
            print(f"  - {dim.replace('_', ' ').title()}: {score}")
        
        print("\n📝 Detailed Dimension Analysis:")
        print("-"*40)
        analysis = result["analysis"]
        for dim, text in analysis.items():
            print(f"\n【{dim.replace('_', ' ').title()}】")
            print(f"  {text}")
    
    print("\n" + "="*100)

# -------------------------
# 8. 保存批量分析结果
# -------------------------
def save_batch_analysis_result(analysis_results: Dict[str, Dict]):
    """保存批量分析结果到JSON文件"""
    timestamp = time.strftime("%Y%m%d_%H%M%S", time.localtime())
    save_path = f"multi_model_poem_analysis_{timestamp}.json"
    
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(analysis_results, f, indent=2, ensure_ascii=False)
    
    print(f"\n✅ 批量分析结果已保存至: {save_path}")
    return save_path

# -------------------------
# 9. 主函数：解析文件 → 批量分析 → 输出 → 保存
# -------------------------
if __name__ == "__main__":
    import time  # 用于生成时间戳
    
    try:
        # 1. 解析多模型诗歌文件
        print("🔍 正在解析多模型诗歌文件...")
        poem_dict = parse_multi_model_poem_file(POEM_FILE_PATH)
        print("✅ 多模型诗歌文件解析完成！")
        
        # 2. 批量分析所有模型的诗歌
        print("\n🚀 开始批量分析所有模型的诗歌...")
        analysis_results = batch_analyze_poems(poem_dict, theme="light through mist, drifting leaves, fading warmth")
        
        # 3. 输出结果
        print_batch_analysis_result(analysis_results, poem_dict)
        
        # 4. 保存分析结果
        save_batch_analysis_result(analysis_results)
        
    except FileNotFoundError as e:
        print(f"❌ {e}")
    except ValueError as e:
        print(f"❌ {e}")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🔄 正在加载本地 Qwen2.5-7B 模型...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]


✅ Qwen2.5-7B 模型加载完成！
🔍 正在解析多模型诗歌文件...
✅ 多模型诗歌文件解析完成！

🚀 开始批量分析所有模型的诗歌...

🚀 开始分析 BASE 模型生成的诗歌...
✅ BASE 模型诗歌分析完成！

🚀 开始分析 SFT 模型生成的诗歌...
✅ SFT 模型诗歌分析完成！

🚀 开始分析 MINDFLOW 模型生成的诗歌...
✅ MINDFLOW 模型诗歌分析完成！

📊 多模型诗歌 LitBench 评分对比分析

🎯 各模型总分对比 (0-10):
------------------------------------------------------------
  • BASE: 8.36
  • SFT: 8.36
  • MINDFLOW: Error

📜 BASE MODEL POEM & ANALYSIS

🖋️ Poem Content:
----------------------------------------
In fields where the sun’s warm hands caress,
And nature whispers to the whispering breeze,
I hear the songs that fill this vast expanse—
Chirping crickets and larks ascending.
Amidst rows of corn and wheat so green,
Where cows graze tranquil as their breaths are seen,
A farmer tills with slow, sure pace and might,
His heart with every furrow taking flight.
Beneath the big blue sky's expansive dome,
He works the earth, the very essence of home.
The soil rich with the promise of tomorrow,
As seasons turn like leaves upon the floor.
This land, so wild 